# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [ ]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [8]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [9]:
ed = Website("https://wfmu.org")
print(ed.title)
print(ed.text)

WFMU-FM 91.1/Jersey City, NJ; 90.1/Hudson Valley, NY – We're an independent freeform station broadcasting at 91.1 fm in New York, at 90.1 fm in the Hudson Valley, and with gobs and gobs of online offerings.
128k MP3
Listen
Sign-in
Register
Signed In |
Schedule + Archive
Toggle navigation
Independent freeform radio broadcasting at 91.1 and 91.9 fm in New Jersey and New York City,
and at 90.1 and 91.9 fm in the Hudson Valley, with gobs of online offerings
Listen
Most Recent Archives
WFMU Live Stream
Rock ‘n’ Soul Radio
Rock ‘n’ Soul Radio Homepage
Give the Drummer Radio
GTDR Homepage
Sheena’s Jungle Room
Sheena’s Jungle Room Homepage
Stream Help
Programming
Program Schedule
Advanced Search
WFMU Mobile
Upcoming Special Programs
Podcasts
Playlists and Program Archives
Heavily Played Records
Send Us Music
Diversions
Monty Hall
Sign Up For Our Monthly Newsletter
Receive WFMU Fan Mail
Listener Photo Gallery
Our Record Fair
WFMU’s Beware of the Blog
Public Service Announcements
About
About WFM

In [10]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [11]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [12]:
print(user_prompt_for(ed))

You are looking at a website titled WFMU-FM 91.1/Jersey City, NJ; 90.1/Hudson Valley, NY – We're an independent freeform station broadcasting at 91.1 fm in New York, at 90.1 fm in the Hudson Valley, and with gobs and gobs of online offerings.
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

128k MP3
Listen
Sign-in
Register
Signed In |
Schedule + Archive
Toggle navigation
Independent freeform radio broadcasting at 91.1 and 91.9 fm in New Jersey and New York City,
and at 90.1 and 91.9 fm in the Hudson Valley, with gobs of online offerings
Listen
Most Recent Archives
WFMU Live Stream
Rock ‘n’ Soul Radio
Rock ‘n’ Soul Radio Homepage
Give the Drummer Radio
GTDR Homepage
Sheena’s Jungle Room
Sheena’s Jungle Room Homepage
Stream Help
Programming
Program Schedule
Advanced Search
WFMU Mobile
Upcoming Special Programs
Podcasts
Playlists and Program Archives
Heavily Played Recor

In [13]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled WFMU-FM 91.1/Jersey City, NJ; 90.1/Hudson Valley, NY – We're an independent freeform station broadcasting at 91.1 fm in New York, at 90.1 fm in the Hudson Valley, and with gobs and gobs of online offerings.\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\n128k MP3\nListen\nSign-in\nRegister\nSigned In |\nSchedule + Archive\nToggle navigation\nIndependent freeform radio broadcasting at 91.1 and 91.9 fm in New Jersey and New York City,\nand at 90.1 and 91.9 fm in the Hudson Valley, with gobs of online offerings\nListen\nMost Recent Archives\nWFMU Live Stream\nRock ‘n’ Soul Radio\nRock ‘n’ Soul Radio Homepag

In [21]:
def summarize(url):
    website = Website(url)

    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(website)
    )
    
    return response.choices[0].message.content

In [22]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [23]:
display_summary('https://wfmu.org')

**Overview**
----------

WFMU (Volunteer FM) is an independent non-commercial radio station serving the New York metropolitan area. The station offers a wide range of programming, including rock, punk, folk, and experimental music.

**Recent News and Updates**

* A new WFMU Archive Bunker App is available for browsing archived shows dating back to 2000.
* The station has announced its plans for the 2025 WFMU Record Fair, which will take place on November 8-9th at the Metropolitan Pavilion in Manhattan.
* Donate your old car or boat to WFMU and receive a tax credit for the book value of your vehicle; call 1-855-WFMU-CAR (855-936-8227) to arrange for pickup.

**Current Programs**

* Rock 'n' Soul Radio
* Give the Drummer Radio
* Sheena's Jungle Room
* WFMU Live Stream

**Upcoming Events and Specials**
---------------------------------

* Various upcoming special programs, including fill-in shifts for regular DJs.
* A two-day event at the Metropolitan Pavilion (record fair dates TBA).

**How to Listen**

* Tune in on-air or stream live on the WFMU website and mobile app.

**Get Involved**

* Donate to support WFMU's mission and activities.
* Become a dealer at the upcoming record fair by emailing [recfair@wfmu.org](mailto:recfair@wfmu.org).
* Follow WFMU on social media for updates, news, and programming announcements.